In [2]:
import nrrd
import numpy as np
import matplotlib as plt

In [3]:
data,header = nrrd.read('annotation_25.nrrd')
print(data.shape)

(528, 320, 456)


In [ ]:
volume_indexes = data>0
# write annotation indexes
volume_indexes.astype('uint8').tofile('./compression/volume_indexes.bytes')

In [ ]:
data_flat = data.flatten()
data_flat = data_flat[volume_indexes.flatten()]
uannotations = np.unique(data_flat)
uannotations.astype('uint32').tofile('./compression/annotation_map.bytes')

In [ ]:
annotation_indexes = np.zeros(data_flat.shape)
for i, uann in enumerate(uannotations):
    annotation_indexes[data_flat==uann] = i
annotation_indexes.astype('uint16').tofile('./compression/annotation_indexes.bytes')

In [ ]:
# Re-load all files!
volume_indexes_load = np.fromfile('./compression/volume_indexes.bytes',dtype='uint8')
print(np.all(volume_indexes.flatten()==volume_indexes_load))
annotation_map_load = np.fromfile('./compression/annotation_map.bytes',dtype='uint32')
print(np.all(uannotations==annotation_map_load))
annotation_indexes_load = np.fromfile('./compression/annotation_indexes.bytes',dtype='uint16')
print(np.all(annotation_indexes==annotation_indexes_load))

In [ ]:
# Reconstruct the annotation volume
ai = 0
i = 0
data_load = np.zeros(data.shape)
for ap in np.arange(data.shape[0]):
    for dv in np.arange(data.shape[1]):
        for lr in np.arange(data.shape[2]):
            if volume_indexes_load[i]==1:
                data_load[ap,dv,lr] = annotation_map_load[annotation_indexes_load[ai]]
                ai+=1
            i+=1
print(np.all(data==data_load))

In [ ]:
# Now build the IBL coverage map
from needles2.probe_model import ProbeModel

res = 25
pr = ProbeModel(res=res)
pr.initialise()
pr.compute_best_for_provenance(provenance='Histology track')
coverage = pr.report_coverage(provenance='Best', dist=354)

In [ ]:
coverage = coverage.swapaxes(1,2)
print(coverage.shape)

In [ ]:
volume_indexes = np.fromfile('./compression/volume_indexes.bytes',dtype='uint8')

In [ ]:
# Go through the flattened file and at each point where data_indexes==1, save whether that was a 0/1/2, we can use the map here this will help let Unity compress the file further
coverage_flat = coverage.flatten()
coverage_map = [0,1,2]

In [ ]:
coverage_indexes = coverage_flat[volume_indexes==1]
print(np.sum(coverage_indexes==0))
print(np.sum(coverage_indexes==1))
print(np.sum(coverage_indexes==2))

In [ ]:
coverage_indexes.astype('uint8').tofile('coverage_indexes.bytes')

In [ ]:
volume_indexes_load = np.fromfile('./compression/volume_indexes.bytes',dtype='uint8')
coverage_indexes_load = np.fromfile('coverage_indexes.bytes',dtype='uint8')

In [ ]:
# Reconstruct the coverage volume
ai = 0
i = 0
cov_load = np.zeros(data.shape)
for ap in np.arange(data.shape[0]):
    for dv in np.arange(data.shape[1]):
        for lr in np.arange(data.shape[2]):
            if volume_indexes_load[i]==1:
                cov_load[ap,dv,lr] = coverage_indexes_load[ai]
                ai+=1
            else:
                cov_load[ap,dv,lr] = np.nan
            i+=1

In [ ]:
print(np.all(coverage==cov_load))

In [ ]:
plt.imshow(cov_load[100,:,:])
plt.imshow(coverage[100,:,:])